# Practice: Spotify's API

This notebook contains a set of exercises that will guide you through the different steps of this practice exercise. Devise solutions that are code-based, i.e. not hard-coded or manually computed. 

<div class="alert alert-success">The aim of this exercise is to create and save a dataset containing information about every song in a given playlist by requesting data from Spotify's API.</div>

## Getting client credentials

Spotify's API uses an Authentication scheme called OAuth. Hence, before starting to make requests, you need to get your client credentials to the Spotify API. 

To do so, you need to have a Spotify account (free or paid). If you don't have one yet, please create a free account before moving on. Once you do, head over to Spotify for Developers, open your [Dashboard](https://developer.spotify.com/dashboard/) and log in with your account. 

Click on “CREATE APP”. Fill in the name and description for your project. Under "Redirect URI", enter "http://localhost:8888/callback" (don't worry about this, as it won't be used). Finally, it the tick boxes, select "Web API". 

Once your App has been created, you should see the app name in the Dashboard. Click on it, then on "Settings", and then on "View Client Secret".  The numbers for “Client ID” and “Client Secret” on correspond to your client credentials.

<div class="alert alert-info">Create two new variables, <b>client_id</b> and <b>client_secret</b>, that store your ID and Key, respectively</div>

In [1]:
client_id = '8758b3dc64314fca8486cfbd03f6b9dc'
client_secret = '61a52a3d65544177a752b3d097e38577'

These are the keys that you will use throughout the remaining of the assignment.



Great! We are good to go. Next step is getting an access token.

## Getting an access token / key

In order to access the various endpoints of the Spotify API, we need to pass an access token. 

To get one, we need to pass a ```POST``` request with our client credentials. This request will create a token resource in the server and respond back with it. We can build this ```POST``` request using ```requests``` library. In other words, the first API call in the spotify API is the call to get the access token. 

<div class="alert alert-info">Run the following cell to built your POST request</div>

In [2]:
import requests

# URL for token resource
auth_url = 'https://accounts.spotify.com/api/token'

# request body
params = {'grant_type': 'client_credentials',
          'client_id': client_id,
          'client_secret': client_secret}

# POST the request
response =  requests.post(url=auth_url,data=params)# YOUR CODE HERE
response.json()

{'access_token': 'BQA-PwCUEp7veDS8SnBxhRAbUMO_mQv2t-L-TRSLOnQUe3m4qiC4ydRwapcYGlOc1O3kewO7jlnwU1INJY9s1gr9JSBBE5d-uRx8DVZFjwwHM--t7ck',
 'token_type': 'Bearer',
 'expires_in': 3600}

Take a look at the response you obtained. The access_token you just retrieved will expire after one hour.

<div class="alert alert-info">Retrieve your token from <b>auth_response</b> and save it in a new variable called <b>access_token</b>. </div>

<div class="alert alert-warning">Make sure you define the <i>access_token</i> variable such that it will be updated each time your code is run from scratch, i.e. make sure it hasn't expired by the time your code is graded.</div>

In [3]:
# YOUR CODE HERE
access_token = response.json()['access_token']
access_token

'BQA-PwCUEp7veDS8SnBxhRAbUMO_mQv2t-L-TRSLOnQUe3m4qiC4ydRwapcYGlOc1O3kewO7jlnwU1INJY9s1gr9JSBBE5d-uRx8DVZFjwwHM--t7ck'

As above, the access token will be used throughout the remaining of the assignment.



This token is your golden ticket to access Spotify's API. A copy of this string is now stored in the server, so that everytime you make a request to the API the server will check that the token you provide and the one it has in store match.

<img src="https://www.dropbox.com/s/hgb02k4h1mtdv22/header.png?raw=1" width="500">

Similar to the Cohere API, Spotify's API expects you to include your access token in the requests header. We won't use headers in this course and this is the only exception: when some APIs require to specify the authentication in the header. Hence, the header has already been formatted for you. 

<div class="alert alert-info">Run the following cell to save the header in a new variable so that you can use it later on.</div>

In [4]:
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

## Poking around

Spotify's API provides numerous endpoints to access things like album listings, artist information, playlists, even Spotify-generated audio analysis of individual tracks, which include their time signature or measurements such as their “danceability” or "loudness". You can take a look at all the information available by reading the [Docs](https://developer.spotify.com/documentation/web-api/reference/). In this assignment you will use several of these endpoints.

In order to get a feel of how the API works, we will begin by making a ```GET``` request to the ```audio-features``` endpoint to extract data for a specific track. In particular, let's retrieve all the information for Radiohead's *Creep* song. 

The first thing you need is to identify the appropriate URL or path to direct your request to. The urls for all Spotify API endpoints follow the same structure. They all use the base URL for the API and are then defined as a concatenation of ```base_url + endpoint```. Sometimes, you will also need to provide some additional information as part of the URL. In the case of ```audio-features```, however, it is enough with just the ```base_url``` and the ```endpoint``` name.

The ```base_url``` is defined below:

In [5]:
base_url = 'https://api.spotify.com/v1'

<div class="alert alert-info">Define the url for the audio-features endpoint by following the instructions above. Store it in a variable called <b>audio_features_endpoint</b>.</div>

In [6]:
audio_features_endpoint = base_url + '/audio-features'

Next thing we need is to fill in the request body. If you check the documentation you'll see that the ```audio-features``` endpoint takes the following query parameters.

<img src="https://www.dropbox.com/s/s4zs6wlue0u16cu/body.png?raw=1" width="500">

Hence, the final thing you need to extract data about Radiohead's Creep song is to locate its ```id```. This is its unique identifier. Spotify has unique ids for tracks, for artists, for albums, for playlists, etc.

![Creep](https://www.dropbox.com/s/kufj6ww2yn069gb/creep.png?raw=1)

You can get the ```id``` for any song by going to Spotify, looking for the song, clicking the “…” by the song name, then “Share” and then “Copy Spotify URI”. 

<div class="alert alert-warning">Note that this procedure also works for retrieving ids for artists, albums or any other resource type. Alternatively, you can also retrieve the id for a song or a playlist by clicking "..." by the name, then "Share" and the "Copy Link". This will return a full link to the resource. The id corresponds to the string right after the last slash sign / and before the ? sign</div>

This URI should be a string that includes something like **spotify:track:**, followed by an alphanumeric sequence. This sequence is the ID you are looking for.

<div class="alert alert-info">Create a new variable called <b>track_id</b> that stores the ID for Radiohead's song Creep.</div>

In [7]:
# YOUR CODE HERE
track_id = '70LcF31zb1H0PyJoS1Sx1r'

Now that you have the id, let's format the URL of the request and send the actual GET request to retrieve the data:

In [8]:
# YOUR CODE HERE
Creep_URL = f'{audio_features_endpoint}/{track_id}'



Now that everything is ready, you can send the actual GET request to retrieve the data.

<div class="alert alert-info">Remember to pass the <i>url</i>, the <i>headers</i> and the <i>params</i> dictionary as arguments to the <i>get</i> function. Convert the response to <i>json</i> format and store it in a new variable called <b>creep</b>.</div>

In [9]:
# YOUR CODE HERE
creep = requests.get(Creep_URL,params=params,headers=headers).json()
creep

{'danceability': 0.515,
 'energy': 0.43,
 'key': 7,
 'loudness': -9.935,
 'mode': 1,
 'speechiness': 0.0372,
 'acousticness': 0.0097,
 'instrumentalness': 0.000133,
 'liveness': 0.129,
 'valence': 0.104,
 'tempo': 91.844,
 'type': 'audio_features',
 'id': '70LcF31zb1H0PyJoS1Sx1r',
 'uri': 'spotify:track:70LcF31zb1H0PyJoS1Sx1r',
 'track_href': 'https://api.spotify.com/v1/tracks/70LcF31zb1H0PyJoS1Sx1r',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/70LcF31zb1H0PyJoS1Sx1r',
 'duration_ms': 238640,
 'time_signature': 4}


<div class="alert alert-warning">Often Spotify has several instances of a track in its catalogue, each available in a different set of markets. This commonly happens when the track the album is on has been released multiple times under different licenses in different markets. These tracks are linked together so that when a user tries to play a track that isn’t available in their own market, the Spotify mobile, desktop, and web players try to play another instance of the track that is available in the user’s market. Even if this issue won't prevent you from being able to play the  song in your device, it might result in different ids for the different markets. Note that the test below is written taking the values for the Spanish market into account.</div>

<div class="alert alert-warning">Note that you can specify the market for this and for future queries through the <i>market</i> parameter.</div>

Congrats! You just made your first successful request to Spotify's API! Feel free to take a look at the information included in the response. Pay special attention to the way in which information is presented. Once you are done, let's move on to some actual work!

## Getting data from a playlist

In the following exercise you will build a dataset containing data about different songs. You can either use a playlist of your own, or use the one mentioned in the documentation.


<div class="alert alert-info">Create a variable called <b>playlist_id</b> that stores the id of your playlist of choice.</div>

<div class="alert alert-warning">Your <i>playlist_id</i> should contain only alphanumeric characters. This means that characters such as ? $ % & / ! should not be included.</div>

In [10]:
playlist_id ='5NFmYXWh6oWM8EvWenFCXd'


In the following you are going to extract information about the different tracks included in this playlist. Hence, let's make sure that the variabl ei spropelry created and that it refers to an actual id.

The next step will be making a request to get full details of the tracks included in your chosen playlist. Remember that you can take a look at all the information available at the different endpoints in Spotify's API by reading the [Docs](https://developer.spotify.com/documentation/web-api/reference/). Locate the right endpoint for your query and read the Docs to find out how to build your request. 

<div class="alert alert-info"><b>Exercise 1 </b>Write the code to retrieve all the items from your chosen playlist from the <i>tracks</i> endpoint. When making your request don't use any of the optional arguments. Direct your request to the right endpoint instead. Store the <i>raw</i> response in a new variable called <i>playlist_response</i>.
</div>

<div class="alert alert-warning">To complete this exercise you must use the requests library</div>

In [11]:
# YOUR CODE HERE
playlist_url = f'{base_url}/playlists/{playlist_id}'
playlist_response = requests.get(f'{playlist_url}/tracks',headers=headers,params=params)


<div class="alert alert-info"><b>Exercise 2 </b>Convert the response to JSON format and store the result in a new variable called <i>playlist</i>.</div>

In [12]:
# YOUR CODE HERE
playlist = playlist_response.json()
print(playlist)

{'href': 'https://api.spotify.com/v1/playlists/5NFmYXWh6oWM8EvWenFCXd/tracks?offset=0&limit=100', 'items': [{'added_at': '2024-10-17T11:42:59Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/ta9grnuodilbjrfec9wuwjws6'}, 'href': 'https://api.spotify.com/v1/users/ta9grnuodilbjrfec9wuwjws6', 'id': 'ta9grnuodilbjrfec9wuwjws6', 'type': 'user', 'uri': 'spotify:user:ta9grnuodilbjrfec9wuwjws6'}, 'is_local': False, 'primary_color': None, 'track': {'preview_url': 'https://p.scdn.co/mp3-preview/7323439c4d11c862403a3d19a1dbcaf725bded9c?cid=8758b3dc64314fca8486cfbd03f6b9dc', 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL'

The following cells contain the tests that will grade your code. **Don't modify them**. Simply leave them as they are.

Take your time to familiarize yourself with the data and how they are presented. Note that, by default, Spotify's API only returns information about a maximum of 100 tracks in a playlist. If your playlist of choice has more that 100 tracks, you'll retrieve the data only for the first 100 of them.

<div class="alert alert-danger">Throghout the following exercises you may come across data that are missing. If so, encode these data using a <b>None</b> (Nonetype), unless otherwise stated. </div>

## Retrieving basic track information

In what follows, you are going to retrieve specific data for each of the tracks contained in your playlist.

<div class="alert alert-info"><b>Exercise 3 </b>Write the code to extract the title (in string form), the name of the album (in string form), the name of the artist (in string form), the duration (in integer form), the track number (in integer form), the popularity (in integer form), and the id (in string form) for all the tracks included in your chosen playlist. Store these data in separate lists called <i>title</i>, <i>album</i>, <i>artist</i>, <i>duration</i>, <i>track_number</i>, <i>track_popularity</i>, and <i>track_id</i>, respectively. In those cases where more than one value is available for these items, retain only the first. In all cases, entries should appear in the same order as they are presented in the playlist.<br></div>

<div class="alert alert-warning">Make sure you correctly set all variable names. They have to be written <b>exactly</b> as given in the instructions.</div>

In [13]:
title = []
album = []
artist = []
duration = []
track_number = []
track_popularity = []
track_id= []
# Loop through the 'items' list to extract the song titles
for item in playlist['items']:
    track = item['track']
    # Add the song title to the list
    if track:  # Check if the track is not None
        title.append(track.get('name', None))
        album.append(track['album'].get('name',None))
        artist.append(track['artists'][0].get('name',None))
        duration.append(track['duration_ms'])
        track_number.append(track['track_number'])
        track_popularity.append(track['popularity'])
        track_id.append(track['id'])

print(f"Title: {title}")
print(f"Album: {album}")
print(f" Artist: {artist}")
print(f"Duration: {duration}")
print(f"Track Number: {track_number}")
print(f"Track Popularity: {track_popularity}")
print(f"Track ID: {track_id}")


Title: ['War', 'CECE', 'All Of A Sudden (feat. Tiffany Hudson & Chris Brown)', 'Yahweh We 🤍 You', 'Sure Been Good (feat. Tiffany Hudson)', 'Iahwe', 'Joy of Salvation - Live', 'Air I Breathe', 'grand prix!', 'Love Like That', 'King Jesus', 'Elohim', 'Yesterday Is Gone', 'ACT', 'At The Cross (Love Ran Red) - Live', 'evergreen love', 'Honey From The Rock', 'New Thing Coming (feat. Tiffany Hudson & Steven Furtick)', 'Ankle Deep - Live', 'Dependency - Live', "I've Tasted and I've Seen - Live", 'He Is Alive', 'Death in Reverse', "It's Okay, It's Alright - Home Recording", 'Digital Death', 'PARADOXOLOGY', 'Voel Jy Die Genade', 'Feels Like A Blessing', 'Matthew 6', 'UP!', 'SUNSHINE', 'My Promised Land', 'Who I Was', 'Altars Over Stages', 'Die a Beggar', 'Take Me Home', 'NEVER LEFT MY SIDE', 'Middle', 'Take Over', 'Hold On Me', 'Thank You Lord', 'Say That I Love You', 'Fishes and Loaves']
Album: ['War', 'L O W B L O W', 'When Wind Meets Fire', 'When Wind Meets Fire', 'When Wind Meets Fire', 'Vi

<div class="alert alert-info"><b>Exercise 4 </b>Write the code to extract the number of available markets (in integer form) for all the tracks included in your chosen playlist. Store these data in a separate list called <i>n_available_markets</i>. As above, entries in this list should appear in the same order as they are presented in the playlist.</div>

In [14]:
n_available_markets = []
for item in playlist['items']:
    track = item['track']
    # Add the song title to the list
    if track:  # Check if the track is not None
        n_available_markets.append(len(track['available_markets']))

print(f"Available markets: {n_available_markets}")

Available markets: [185, 185, 185, 185, 185, 185, 185, 185, 183, 185, 185, 185, 185, 185, 184, 185, 185, 185, 185, 185, 185, 185, 0, 185, 185, 185, 184, 183, 185, 184, 184, 185, 185, 185, 185, 185, 184, 185, 185, 185, 185, 185, 185]


Each track in the playlist is associated to an album. Let's retrieve the corresponding release date.

<div class="alert alert-info"><b>Exercise 5 </b>Write the code to extract the release year (in int form) and month (in string form) for all the tracks included in your chosen playlist. Store these data in separate lists called <i>release_year</i> and <i>release_month</i>, respectively. Entries should appear in these lists in the same order as they are presented in the playlist.</div>

<div class="alert alert-warning">Make sure you correctly set the variable names. They have to be written <b>exactly</b> as given in the instructions.</div>

<div class="alert alert-warning">Months should be stored with their full names and capitalized: use <i>September</i> for 09, <i>January</i> for 01, etc.</div>

In [15]:
import datetime
import calendar

release_date = []
release_year = []
release_month = []
for item in playlist['items']:
    track = item['track']
    # Add the song title to the list
    if track:  # Check if the track is not None
        release_date.append(track['album'].get('release_date',None))

dates = [i.split('-') for i in release_date]
release_year = [row[0] for row in dates]
months_num = [row[1] for row in dates]

release_months = [calendar.month_name[int(month)] for month in months_num]
print(f"Release Year: {release_year}")
print(f"Release Month: {release_months}")


Release Year: ['2024', '2023', '2024', '2024', '2024', '2023', '2024', '2023', '2024', '2024', '2023', '2023', '2023', '2023', '2014', '2024', '2023', '2024', '2023', '2023', '2023', '2024', '2023', '2021', '2021', '2022', '2019', '2022', '2023', '2024', '2024', '2024', '2024', '2024', '2024', '2024', '2024', '2022', '2023', '2022', '2021', '2022', '2023']
Release Month: ['March', 'August', 'July', 'July', 'July', 'January', 'August', 'May', 'May', 'September', 'August', 'May', 'August', 'August', 'January', 'June', 'February', 'July', 'December', 'January', 'November', 'February', 'October', 'February', 'October', 'September', 'February', 'November', 'July', 'July', 'July', 'May', 'May', 'May', 'May', 'May', 'July', 'July', 'March', 'July', 'December', 'July', 'June']


## Retrieving additional track information

In what follows, you are going to retrieve additional data for each of the tracks contained in your playlist.

<div class="alert alert-info"><b>Exercise 6 </b>Write the code to extract data about the danceability, energy, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence and tempo for all the tracks included in your chosen playlist. Store these data in separate lists called <i>danceability</i> (in float form), <i>energy</i>, <i>loudness</i> (in float form), <i>mode</i> (in int form), <i>speechiness</i> (in float form), <i>acousticness</i> (in float form), <i>instrumentalness</i> (in int and float form), <i>liveness</i> (in float form), <i>valence</i> (in float form) and <i>tempo</i> (in float form), respectively. In those cases where more than one value is available for these items, retain only the first. In all cases, entries should appear in the same order as they are presented in the playlist.</div>

In [16]:
# YOUR CODE HERE
# danceability (in float form),energy, loudness (in float form),mode (in int form), speechiness (in float form),acousticness (in float form),instrumentalness(in int and float form), liveness (in float form), valence (in float form), tempo
audio_features = {}
danceability = []
energy = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []

for song in track_id:
    url = f'{audio_features_endpoint}/{song}' 
    audio_feat = requests.get(url,params=params,headers=headers).json()
    danceability.append(float(audio_feat['danceability']))
    energy.append(float(audio_feat['energy']))
    loudness.append(float(audio_feat['loudness']))
    mode.append(int(audio_feat['mode']))
    speechiness.append(float(audio_feat['speechiness']))
    acousticness.append(float(audio_feat['acousticness']))
    instrumentalness.append(audio_feat['instrumentalness'])
    liveness.append(float(audio_feat['liveness']))
    valence.append(float(audio_feat['valence']))
    tempo.append(float(audio_feat['tempo']))


print(f"Danceablity: {danceability}") 
print(f"Energy: {energy}") 
print(f"Loudness: {loudness}") 
print(f"Mode: {mode}")
print(f"Speechiness: {speechiness}")
print(f"Acousticness: {acousticness}")
print(f"Instrumentalness {instrumentalness}")
print(f"Liveness: {liveness}")
print(f"Valence: {valence}")
print(f"Tempo {tempo}")


Danceablity: [0.737, 0.886, 0.286, 0.472, 0.513, 0.38, 0.347, 0.436, 0.55, 0.736, 0.769, 0.79, 0.768, 0.854, 0.316, 0.564, 0.855, 0.235, 0.298, 0.345, 0.282, 0.345, 0.733, 0.631, 0.51, 0.421, 0.735, 0.503, 0.376, 0.79, 0.716, 0.693, 0.617, 0.529, 0.632, 0.671, 0.393, 0.684, 0.743, 0.595, 0.714, 0.612, 0.465]
Energy: [0.574, 0.679, 0.557, 0.702, 0.575, 0.629, 0.627, 0.561, 0.844, 0.837, 0.681, 0.762, 0.498, 0.574, 0.581, 0.578, 0.408, 0.831, 0.804, 0.354, 0.923, 0.703, 0.358, 0.245, 0.596, 0.368, 0.858, 0.676, 0.691, 0.782, 0.536, 0.627, 0.215, 0.525, 0.404, 0.821, 0.0461, 0.618, 0.43, 0.591, 0.44, 0.535, 0.404]
Loudness: [-6.71, -6.503, -8.128, -7.509, -8.539, -10.211, -9.749, -8.247, -4.348, -7.902, -7.809, -6.537, -10.442, -7.149, -6.609, -4.987, -12.616, -7.435, -12.903, -10.909, -8.878, -6.56, -9.315, -9.9, -8.116, -9.507, -4.834, -7.352, -4.307, -6.293, -5.675, -6.048, -9.173, -5.643, -6.148, -3.356, -19.779, -7.678, -12.154, -7.147, -10.676, -8.411, -6.357]
Mode: [0, 1, 1, 1, 1, 

## Retrieving artist information

In what follows, you are going to retrieve data about the artists for each of the tracks contained in your playlist.

<div class="alert alert-info"><b>Exercise 7 </b>Write the code to extract data about the total number of followers (in int form), the first listed genre (in string form) and the popularity (in int form) for the artists of all the tracks included in your chosen playlist. Store these data in separate lists called <i>artist_followers</i>, <i>genre</i> and <i>artist_popularity</i>. In cases where no genre is given, fill in the corresponding entry using a None.</div>

<div class="alert alert-warning">Note that the artist information has to to be extracted from the tracks themselves.</div>

In [17]:
# YOUR CODE HERE
artist_id = []
artist_followers =[]
genre = []
artist_popularity = []

for item in playlist['items']:
    track = item['track']
    artist_id = track['artists'][0].get('id',None)
    url = f"{base_url}/artists/{artist_id}"
    artist_info = requests.get(url,params=params,headers=headers).json()
    artist_followers.append(artist_info.get('followers',{}).get('total',None))
    artist_popularity.append(artist_info.get('popularity',None))
    all_genres = artist_info.get('genres',None)
    if not all_genres:
        genre.append(None)
    else:
        genre.append(all_genres[0])
    

print(f"The artists have the following followers :{artist_followers}")
print(f"The artists have the following popularity: {artist_popularity}")
print(f"These are the genres: {genre}")

The artists have the following followers :[257410, 48878, 3889635, 3889635, 3889635, 3005, 248, 159713, 72286, 391382, 566511, 159713, 257410, 48878, 455467, 79577, 15762, 3889635, 4029, 4029, 4029, 1370, 10219, 1770, 107269, 5800, 193354, 67704, 29132, 827536, 827536, 266352, 266352, 266352, 266352, 266352, 827536, 191114, 191114, 191114, 191114, 191114, 266352]
The artists have the following popularity: [63, 51, 77, 77, 77, 31, 8, 58, 57, 67, 63, 58, 63, 51, 63, 53, 44, 77, 25, 25, 25, 23, 33, 16, 61, 41, 41, 62, 50, 74, 74, 69, 69, 69, 69, 69, 74, 54, 54, 54, 54, 54, 69]
These are the genres: [None, 'christian hip hop', 'ccm', 'ccm', 'ccm', 'muzica crestina', None, None, None, None, 'christian hip hop', None, None, 'christian hip hop', 'ccm', None, 'christian lo-fi', 'ccm', None, None, None, None, 'christian indie', 'christian indie folk', 'christian pop', 'pop worship', 'afrikaans', 'ccm', None, None, None, 'ccm', 'ccm', 'ccm', 'ccm', 'ccm', None, 'christian afrobeat', 'christian a

In addition to the above, there's also additional information we can retrieve about each artist. For this purpose, let's first retrieve the list of distinct artists for the different tracks in your playlist.

<div class="alert alert-info"><b>Exercise 8 </b>Write the code to identify the list of unique artist ids that correspond to the different tracks in your chosen playlist. Store these data in a list called <i>unique_artist_ids</i>.</div>

<div class="alert alert-warning">Retrieve the artist ids that correspod directly to the tracks.</div>

In [18]:
unique_artist_ids = []

for item in playlist['items']:
    track = item['track']
    unique_artist_ids.append(track['artists'][0].get('id',None))

print(f" the uniques artist ID's are: {unique_artist_ids}")

 the uniques artist ID's are: ['6s3XaJkcT7464G4oII9V41', '1sPm31qmcbk9EFoRCS8eRl', '3YCKuqpv9nCsIhJ2v8SMix', '3YCKuqpv9nCsIhJ2v8SMix', '3YCKuqpv9nCsIhJ2v8SMix', '69DVsCHh39tZuwkvw2IIiU', '479IvQ8CYPgzCJ1fuRbsPD', '2PCl4hx9Qds8sjcnoWUdqp', '5GevjOibFX6uwZ1mTYfhbn', '3zSrc5vUlUxyDdS0KrxFJO', '77IKXFvO7SpWrq8hflrUXc', '2PCl4hx9Qds8sjcnoWUdqp', '6s3XaJkcT7464G4oII9V41', '1sPm31qmcbk9EFoRCS8eRl', '6piIAIurGAryW5h1rqQC16', '5lHDypXbNmHTDoFWpSTqXd', '0RylJjG2CawOJWWxRD6Qts', '3YCKuqpv9nCsIhJ2v8SMix', '2yCeqTo1EY9pC0PiimyCqd', '2yCeqTo1EY9pC0PiimyCqd', '2yCeqTo1EY9pC0PiimyCqd', '5sWkuThhqRdUFwy7faCWoH', '58aCPqTdL2wim0FOUm2txg', '3e4wJ7a4uWZfGilll3TiAt', '31Bh5G3Q9JzwqPpHDLIQtW', '2YrmP8ksRyeEVkzYp5to25', '0Zg9ZuI494uRe8eMGTbZLs', '6CK4CBL0n07VdVZmfh40nm', '7Fqu2gBrthnYE53IGOltaI', '1scVfBymTr3CeZ4imMj1QJ', '1scVfBymTr3CeZ4imMj1QJ', '7yAymwuYh9RcpX84GSNIq2', '7yAymwuYh9RcpX84GSNIq2', '7yAymwuYh9RcpX84GSNIq2', '7yAymwuYh9RcpX84GSNIq2', '7yAymwuYh9RcpX84GSNIq2', '1scVfBymTr3CeZ4imMj1QJ', '3VY7Il

We are now interested in retrieving catalog information about each artist’s top tracks. This information is provided by Spotify's API on a country basis. Here, we will retrieve the information corresponding to Spain, whose *ISO 3166-1 alpha-2* code is **ES**. The information we are looking for is stored under the ```top-tracks``` endpoint for ```artists```. Requests to this location retrieve the 10 most famous tracks for a given artist id.

<div class="alert alert-info"><b>Exercise 9 </b>Write the code to retrieve the 10 top tracks for each of the unique artists in your chosen playlist. Store these data in a dictionary called <i>top_tracks</i>. The keys of this dictionary should correspond to the unique artist ids stored in the list <i>unique_artist_id</i>. The values of this dictionary should include the names of the 10 most popular songs in a list.</div>

<div class="alert alert-warning">In cases where the number of tracks included in the top-tracks endpoint is below 10, simply retrieve the provided list.</div>


In [19]:
top_tracks ={}
tracks = []
for name in unique_artist_ids:
    url = f"{base_url}/artists/{name}/top-tracks?market=ES" # make the market Spaion and retreive the top tracks for each artist
    toets = requests.get(url,params=params,headers=headers).json()
    for item in toets['tracks']:
        tracks.append(item.get('name', None))
    top_tracks[name] = tracks # add the track name to top_tracks
    tracks = []
        
print(f"The top tracks are:{top_tracks}")

The top tracks are:{'6s3XaJkcT7464G4oII9V41': ['GOD IS GOOD', 'War', 'Rocky Road, Pt.2', 'Yesterday Is Gone', 'Fruits of My Labor', 'Old Ways', 'Psalms 139', 'Way Too Blessed', 'Quick', 'I Choose You'], '1sPm31qmcbk9EFoRCS8eRl': ['STILL HOLY', 'ACT', 'TR!BE ON THE MOVE', 'RED FLAGS', 'PULP FICTION', 'CECE', 'MANDATORY', 'OUTSIDE', 'RICH', 'TEAM'], '3YCKuqpv9nCsIhJ2v8SMix': ['Praise (feat. Brandon Lake, Chris Brown & Chandler Moore)', 'O Come to the Altar (Live)', 'Trust In God (feat. Chris Brown)', 'Jireh (feat. Chandler Moore & Naomi Raine)', 'Graves Into Gardens (Live)', 'The Blessing (Live)', 'Same God (feat. Jonsal Barrientes)', 'LION (feat. Chris Brown & Brandon Lake)', 'Do It Again', 'Been So Good (feat. Tiffany Hudson)'], '69DVsCHh39tZuwkvw2IIiU': ['Cântarea Lui Ghedeon', 'Iahwe', 'Vino La Noi', 'Profet', 'Bun', 'Numele tău', 'Vrednic', 'Abba', 'Botez cu Foc', 'Iahwe'], '479IvQ8CYPgzCJ1fuRbsPD': ['Joy of Salvation - Live', 'Better Than Life (Psalm 63)'], '2PCl4hx9Qds8sjcnoWUdqp'

We can now use this information to identify those songs in your chosen playlist that correspond to each artist's top tracks.

<div class="alert alert-info"><b>Exercise 10 </b>Write the code to check whether the different tracks in your chosen playlist are included among the corresponding artist's top tracks. Store the results in a list called <i>is_top</i>. This list should include the boolean value <i>True</i> whenever the considered track is among the top tracks for that artist and <i>False</i> otherwise.</div>

<div class="alert alert-warning">When comparing track names, consider only <b>exact</b> matches.</div>

In [20]:
# YOUR CODE HERE
is_top = []
for ind in list(range(len(title))):
    if title[ind] in top_tracks[unique_artist_ids[ind]]:
        is_top.append(True)
    else:
        is_top.append(False)

print(f"The results are: {is_top}")

The results are: [True, True, False, False, False, True, True, True, True, True, True, True, True, True, False, True, True, False, True, True, True, True, False, True, False, True, True, False, True, True, False, True, False, True, True, False, False, False, False, False, True, True, True]


A very interesting feature in Spotify is the song recommender.

When creating a playlist from scratch, Spotify offers help by suggesting additional tracks that may be related to the ones already added. Spotify's API also provides a specific endpoint for `recommendations`. We will use this endpoint to retrieve one recommendation for each song in our original playlist.

<div class="alert alert-info"><b>Exercise 11 </b>Write the code to retrieve the id, title and artist from the list of recommendations for each track in your playlist. Store these data in new lists called <i>recommendatin_id</i>, <i>recommendation_title</i> and <i>recommendation_artist</i>.</div>

<div class="alert alert-warning">Retrieve recommendations based solely on each track.</div>

<div class="alert alert-warning">Retrieve information only about the first track in the recommendations list.</div>

<div class="alert alert-warning">When more than one artist is associated to the recommended track, retrieve information only about the first.</div>

In [21]:
url = f"{base_url}/recommendations"
recommendation_id = []
recommendation_title = []
recommendation_artist = []

for item in range(len(title)):
    track = title[item]
    idnum = track_id[item]
    params = {'seed_tracks': idnum,'limit': 1}
    response = requests.get(url, params=params, headers=headers) 
    if response.status_code == 200:
        print('Hello')
        recommend = response.json()
        rec_track = recommend['tracks'][0]
        if rec_track:
            recommendation_id.append(rec_track['id'])
            recommendation_title.append(rec_track['name'])
            recommendation_artist.append(rec_track['artists'][0]['name'])
    else:
        print(f"Error: {response.status_code} - {response.text}")
    
print(f"Recommendation Titles: {recommendation_title}")
print(f"Recommendation IDs {recommendation_id}")
print(f"Recommendation Artists: {recommendation_artist}")

Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 429 - Too many requests
Error: 4

In [22]:
# YOUR CODE HERE

## Bonus exercise

You can complete the exercises below to obtain extra points. These points will be added to your score directly. The maximum score in this assignment is a 10.

<div class="alert alert-danger"><b>Bonus 1 </b>Write the code to find the most popular song in your playlist. If several songs have the same popularity, choose the one for which the artist has the most followers. Save the name of the song to a new variable called <i>most_popular</i>. This variable should <b>only</b> contain the name of the most popular song in string form.</div>

In [23]:
max_popularity = max(track_popularity)
matches = []
count = 0

for ind in track_popularity:
    if ind == max_popularity:
        matches.append(count)
    count += 1

art_pop = [artist_popularity[i] for i in matches]
max_art = max(art_pop)
count = 0

for ind in art_pop:
    if ind == max_art:
        index_f = count
    count += 1

most_popular = title[matches[index_f]]
print(f"The most popular song on the playlist is {most_popular}")


The most popular song on the playlist is UP!


You might have noticed that there exist slight incosistencies in the way in which the track title are included in the playlist, versus the top 10 artist tracks. For example, the fourht song in the provided playlist appears as <i>High And Dry</i> within the title of the playlist songs but as <i>High and Dry</i> (with a small "a" for the word "and") in the Radiohead's top tracks. This results in the fact that the song of the playlist will not be included as one of the artist's top tracks in the is_top list.

<div class="alert alert-danger"><b>Bonus 2 </b>Write the code to create a new list called <i>is_top_2</i>. This time make sure that you look for partial matches of title names too.</div>

In [24]:
# YOUR CODE HERE
is_top_2 = []
for ind in range(len(title)):
    if any(title[ind].lower() in track.lower() for track in top_tracks[unique_artist_ids[ind]]):
        is_top_2.append(True)
    else:
        is_top_2.append(False)

print(f"The results are: {is_top_2}")

The results are: [True, True, False, False, False, True, True, True, True, True, True, True, True, True, False, True, True, False, True, True, True, True, True, True, False, True, True, False, True, True, False, True, False, True, True, False, False, False, False, False, True, True, True]
